In [1]:
import pandas as pd

from master_thesis.core.utils.reproducibility import seed_everything
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt

from master_thesis.core.methods.tcav import TCAV
from master_thesis.core.probes import load_cav
from master_thesis.core.utils.reproducibility import save_results


DEVICE = "cuda"
PROMPT_TYPE = "few_shot"
POSITIVE_TOKEN = "true"
NEGATIVE_TOKEN = "false"
MODEL = "LLAMA_2_7B_CHAT"

LABEL_ASPECT_MODEL = "MMProbe"
LABEL_ASPECT_LAYER = 12

CONFOUNDING_ASPECT_MODEL = "MMProbe"
CONFOUNDING_ASPECT_LAYER = 12


DATA_DIR = "../../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/car_vs_bike/test"
CACHE_DIR = f".cache/results"
CLASS_NAMES = ["bike", "car"]
LABEL_ASCPECT_NAMES = ["bike", "car"]
CONFOUNDING_ASPECT_NAMES = ["negative", "positive"]

In [2]:
seed_everything()

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
TOKENS = [tokenizer.encode(NEGATIVE_TOKEN)[-1], tokenizer.encode(POSITIVE_TOKEN)[-1]]

### Prepare datasets, prompts and cavs

In [5]:
test_50_50 = pd.read_csv(f"{DATASETS_DIR}/test_50_50.csv")
test_95_5 = pd.read_csv(f"{DATASETS_DIR}/test_95_5.csv")

In [6]:
label_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/car_vs_bike",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="label_aspect",
)

confounding_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/car_vs_bike",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="confounding_aspect",
)

In [7]:
label_cav = load_cav(f".cache/cavs/{LABEL_ASPECT_MODEL}", f"label_{LABEL_ASPECT_LAYER}")
confounding_cav = load_cav(
    f".cache/cavs/{CONFOUNDING_ASPECT_MODEL}", f"confounding_{CONFOUNDING_ASPECT_LAYER}"
)

### Estimate TCAV

### Label

#### Test 50 50

In [16]:
seed_everything()

tcav = TCAV(
    cav=label_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    prompt=label_prompt,
    layer=12,
    device=DEVICE,
    verbose=True,
)

In [ ]:
test_50_50_results_label = tcav.evaluate(test_50_50, aspect_label_name="label")

In [3]:
save_results(
    test_50_50_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/tcav/test_50_50/label_aspect_{LABEL_ASPECT_MODEL}_{LABEL_ASPECT_LAYER}.json",
)

In [4]:
test_50_50_results_label

{0: {0: 0.0, 1: 0.01}, 1: {0: 0.0, 1: 0.99}}

#### Test 95 5

In [12]:
seed_everything()

tcav = TCAV(
    cav=label_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    prompt=label_prompt,
    layer=12,
    device=DEVICE,
    verbose=True,
)

In [ ]:
test_95_5_results_label = tcav.evaluate(test_95_5, aspect_label_name="label")

In [6]:
save_results(
    test_95_5_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/tcav/test_95_5/label_aspect_{LABEL_ASPECT_MODEL}_{LABEL_ASPECT_LAYER}.json",
)

In [7]:
test_95_5_results_label

{0: {0: 0.0, 1: 0.0}, 1: {0: 0.0, 1: 0.99}}

### Confounding

#### Test 50 50

In [13]:
seed_everything()

tcav = TCAV(
    cav=confounding_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    prompt=label_prompt,
    layer=12,
    device=DEVICE,
    verbose=True,
)

In [ ]:
test_50_50_results_confounding = tcav.evaluate(
    test_50_50, aspect_label_name="confounding"
)

In [9]:
save_results(
    test_50_50_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/tcav/test_50_50/confounding_aspect_{CONFOUNDING_ASPECT_MODEL}_{CONFOUNDING_ASPECT_LAYER}.json",
)

In [10]:
test_50_50_results_confounding

{0: {0: 0.48, 1: 0.5}, 1: {0: 0.0, 1: 1.0}}

#### Test 95 5

In [9]:
seed_everything()

tcav = TCAV(
    cav=confounding_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    prompt=label_prompt,
    layer=12,
    device=DEVICE,
    verbose=True,
)

In [ ]:
test_95_5_results_confounding = tcav.evaluate(
    test_95_5, aspect_label_name="confounding"
)

In [12]:
save_results(
    test_95_5_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/tcav/test_95_5/confounding_aspect_{CONFOUNDING_ASPECT_MODEL}_{CONFOUNDING_ASPECT_LAYER}.json",
)

In [13]:
test_95_5_results_confounding

{0: {0: 0.05, 1: 0.05}, 1: {0: 0.0, 1: 1.0}}